In [1]:
# Packages

import pandas as pd
import numpy as np
import os
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

from typing import Dict

In [2]:
data_path = os.path.join('./data')
train_file_name = 'train.csv'
test_file_name = 'test.csv'

In [3]:
# Data Load

train = pd.read_csv(os.path.join(data_path, train_file_name))
test = pd.read_csv(os.path.join(data_path, test_file_name))

In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
train.shape

(1460, 81)

In [7]:
test.shape

(1459, 80)

In [8]:
categorical = train.select_dtypes(include=['object', 'category']).columns.to_list()
categorical

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [9]:
def analyze_missing_values(df: pd.DataFrame) -> Dict[str, Dict]:
    result = {}
    missing_cols = df.columns[df.isnull().any()].tolist()

    for col in missing_cols:
        missing_ratio = df[col].isnull().mean() * 100
        dtype = str(df[col].dtype)

        unique_values = df[col].dropna().unique()
        if len(unique_values) > 10:
            unique_values = f"Too many unique values ({len(unique_values)})"
        
        result[col] = {
            'missing_ratio': missing_ratio,
            'dtype': dtype,
            'unique_values': unique_values
        }
    
    return result

def data_missing_value_analysis(df_train: pd.DataFrame, df_test: pd.DataFrame):
    print("\nAnalyzing Train Data")
    print("=" * 50)
    missing_info = analyze_missing_values(df_train)
    for col, info in missing_info.items():
        print(f"Column: {col}")
        print(f"Missing Ratio: {info['missing_ratio']:.2f}%")
        print(f"Data Type: {info['dtype']}")
        print(f"Unique Values: {info['unique_values']}")
        print()
    # Analyze test data
    print("\nAnalyzing Test Data")
    print("=" * 50)
    missing_info = analyze_missing_values(df_test)
    for col, info in missing_info.items():
        print(f"Column: {col}")
        print(f"Missing Ratio: {info['missing_ratio']:.2f}%")
        print(f"Data Type: {info['dtype']}")
        print(f"Unique Values: {info['unique_values']}")
        print()
    return

In [10]:
data_missing_value_analysis(train, test)


Analyzing Train Data
Column: LotFrontage
Missing Ratio: 17.74%
Data Type: float64
Unique Values: Too many unique values (110)

Column: Alley
Missing Ratio: 93.77%
Data Type: object
Unique Values: ['Grvl' 'Pave']

Column: MasVnrType
Missing Ratio: 59.73%
Data Type: object
Unique Values: ['BrkFace' 'Stone' 'BrkCmn']

Column: MasVnrArea
Missing Ratio: 0.55%
Data Type: float64
Unique Values: Too many unique values (327)

Column: BsmtQual
Missing Ratio: 2.53%
Data Type: object
Unique Values: ['Gd' 'TA' 'Ex' 'Fa']

Column: BsmtCond
Missing Ratio: 2.53%
Data Type: object
Unique Values: ['TA' 'Gd' 'Fa' 'Po']

Column: BsmtExposure
Missing Ratio: 2.60%
Data Type: object
Unique Values: ['No' 'Gd' 'Mn' 'Av']

Column: BsmtFinType1
Missing Ratio: 2.53%
Data Type: object
Unique Values: ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' 'LwQ']

Column: BsmtFinType2
Missing Ratio: 2.60%
Data Type: object
Unique Values: ['Unf' 'BLQ' 'ALQ' 'Rec' 'LwQ' 'GLQ']

Column: Electrical
Missing Ratio: 0.07%
Data Type: object
Unique

In [11]:
def simple_preprocess(df_train: pd.DataFrame, df_test: pd.DataFrame):
    df_train_copy = df_train.copy()
    df_test_copy = df_test.copy()
    missing_analysis = analyze_missing_values(df=df_train)

    # 결측치가 50% 이상인 컬럼 삭제
    drop_cols = [ col for col, info in missing_analysis.items() if info['missing_ratio'] >= 50 ]
    df_train_copy.drop(drop_cols, axis = 1, inplace = True)
    df_test_copy.drop(drop_cols, axis = 1, inplace = True)

    categorical = df_train_copy.select_dtypes(include=['object']).columns.to_list()
    
    for col in df_train_copy.columns[df_train_copy.isnull().any()].tolist():
        if col in categorical:
            df_train_copy[col].fillna('Un', inplace=True) # Unknown
            df_test_copy[col].fillna('Un', inplace = True)
        else:
            median_value = df_train_copy[col].median()
            df_train_copy[col].fillna(value=median_value, inplace=True)
            df_test_copy[col].fillna(value=median_value, inplace=True)

    for col in categorical:
        le = LabelEncoder()
        le = le.fit(df_train_copy[col])
        df_train_copy[col] = le.transform(df_train_copy[col])
        
        for val in df_test_copy[col].unique():
            if val not in le.classes_:
                le.classes_ = np.append(le.classes_, val)
        df_test_copy[col] = le.transform(df_test_copy[col])

    return (df_train_copy, df_test_copy)

In [12]:
X_train = train.drop('SalePrice', axis = 1)
y_train = train['SalePrice']
X_test = test

In [13]:
X_train, X_test = simple_preprocess(X_train, X_test)

/var/folders/ws/yk5_j0490vnb543l1m2l7mtc0000gn/T/ipykernel_84932/2234915327.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_copy[col].fillna(value=median_value, inplace=True)
/var/folders/ws/yk5_j0490vnb543l1m2l7mtc0000gn/T/ipykernel_84932/2234915327.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin

In [14]:
train_analysis = analyze_missing_values(train)
drop_cols = [ col for col, info in train_analysis.items() if info['missing_ratio'] >= 50 ]
drop_cols

['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']

In [15]:
train.isnull().sum().sort_values(ascending=False)

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
MasVnrType      872
               ... 
ExterQual         0
Exterior2nd       0
Exterior1st       0
RoofMatl          0
SalePrice         0
Length: 81, dtype: int64

In [16]:
test.isnull().sum().sort_values(ascending=False)

PoolQC           1456
MiscFeature      1408
Alley            1352
Fence            1169
MasVnrType        894
                 ... 
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
SaleCondition       0
Length: 80, dtype: int64

In [17]:
train.describe(exclude='object')

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [20]:
test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [21]:
X_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,2,80.0,11622,1,3,3,0,4,...,0,0,0,120,0,0,6,2010,8,4
1,1462,20,3,81.0,14267,1,0,3,0,0,...,36,0,0,0,0,12500,6,2010,8,4
2,1463,60,3,74.0,13830,1,0,3,0,4,...,34,0,0,0,0,0,3,2010,8,4
3,1464,60,3,78.0,9978,1,0,3,0,4,...,36,0,0,0,0,0,6,2010,8,4
4,1465,120,3,43.0,5005,1,0,1,0,4,...,82,0,0,144,0,0,1,2010,8,4


In [22]:
y_test = X_test['Id']

In [23]:
X_test.drop('Id', axis = 1, inplace = True)

In [24]:
X_train.drop('Id', axis = 1, inplace = True)

In [25]:
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [26]:
y_pred = model.predict(X_test)

In [27]:
y_df = pd.DataFrame()
y_df['Id'] = y_test
y_df['SalePrice'] = y_pred
y_df

,Id,SalePrice
0,1461,123779.226562
1,1462,153149.468750
2,1463,190687.093750
3,1464,192260.093750
4,1465,187176.343750
...,...,...
1454,2915,74617.976562
1455,2916,72853.156250
1456,2917,170424.921875
1457,2918,111800.648438


In [28]:
y_df.columns

Index(['Id', 'SalePrice'], dtype='object')

In [29]:
y_df.to_csv('./my_prediction.csv', index=False)